In [15]:
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
import numpy as np
from pprint import pprint
import requests
from collections import defaultdict

In [2]:
files = os.listdir("scraping/MEPs")

In [3]:
files[:5]

['ADAM_GIEREK_home.html',
 'ADAM_KOSA_home.html',
 'ADAM_SZEJNFELD_home.html',
 'ADINA-IOANA_VALEAN_home.html',
 'AGNES_JONGERIUS_home.html']

In [65]:
files.index("ANNA+ELZBIETA_FOTYGA_home.html")

42

In [164]:
with open("scraping/MEPs/"+files[85], encoding='utf-8') as f:
    rd = f.read()

In [165]:
soup = BeautifulSoup(rd, "lxml")

In [166]:
name_pattern = r'<li class="mep_name"><a id="mep_name_button" class="listen" title="(.*?)">.*?<br/>'
re.findall(name_pattern, rd)

['Bill ETHERIDGE']

In [167]:
soup.findAll('img', {"class":"photo_mep"})[0].attrs["title"]

'Bill ETHERIDGE'

In [168]:
group = soup.findAll('li', {"class":"group"})[0].text.strip()
group

'Europe of Freedom and Direct Democracy Group'

In [169]:
country = soup.findAll('li', {"class":"nationality noflag"})[0].text.strip().split()
country

['United', 'Kingdom', 'United', 'Kingdom', 'Independence', 'Party']

In [170]:
committee = [x.strip() for x in soup.findAll('ul', {"class":"events_collection questeur"}
                                            )[0].text.strip().split("\n") if len(x.strip())]
committee

IndexError: list index out of range

In [171]:
soup.findAll('ul', {"class":"events_collection questeur"})

[]

In [120]:
emailline = soup.findAll('a', {"title":"Follow this MEP"})[0]
handle = emailline.attrs['href'].split("/")[-2]

email = handle + "@europarl.europa.eu"
email

'adinaioana.valean@europarl.europa.eu'

In [125]:
phone = soup.findAll('span', {"class":"phone"})[0].text.strip()
phone = re.sub("\s+", " ", phone)
phone

'+32(0)2 28 45861'

In [138]:
twitter = soup.findAll('a', {"class":"link_twitt"})[1].attrs['href']
twitter

'https://twitter.com/AdinaValean'

In [ ]:
facebook = soup.findAll('a', {"class":"link_fb"})[1].attrs['href']
facebook

In [172]:
def find_and_return(file_name):
    
    with open("scraping/MEPs/"+file_name, encoding='utf-8') as f:
        rd = f.read()
        
    soup = BeautifulSoup(rd, "lxml")
    
    try:
        facebook = soup.findAll('a', {"class":"link_fb"})[1].attrs['href']
    except:
        facebook = -999
    try:
        twitter = soup.findAll('a', {"class":"link_twitt"})[1].attrs['href']
    except:
        twitter = -999
    phone = re.sub("\s+", " ", soup.findAll('span', {"class":"phone"})[0].text.strip())

    emailline = soup.findAll('a', {"title":"Follow this MEP"})[0]
    handle = emailline.attrs['href'].split("/")[-2]
    email = handle + "@europarl.europa.eu"
    
    try:
        committee = [x.strip() for x in soup.findAll('ul', {"class":"events_collection questeur"}
                                            )[0].text.strip().split("\n") if len(x.strip())]
    except:
        committee = -999
        
    country = soup.findAll('li', {"class":"nationality noflag"})[0].text.strip().split()
    group = soup.findAll('li', {"class":"group"})[0].text.strip()

    name = soup.findAll('img', {"class":"photo_mep"})[0].attrs["title"]
    
    return name, group, country, committee, email, phone, twitter, facebook

In [189]:
df = pd.DataFrame(columns="name, group, country, committee, email, phone, twitter, facebook".split(', '))

for i,file in enumerate(files):
    name, group, country, committee, email, phone, twitter, facebook = find_and_return(file)
    
    df = df.append({v:k for k,v in zip([name, group, country, committee, email, phone, twitter, facebook],
                                 "name, group, country, committee, email, phone, twitter, facebook".split(', '))},
              ignore_index=True)
    
#     print(i, "\n")
#     print("Name: {}\nGroup: {}\nCountry: {}\nCommittee: {}\nEmail: {}\nPhone: {}\nTwitter: {}\nFacebook: {}\n".format(
#         name, group, country, committee, email, phone, twitter, facebook))
#     print()

In [190]:
df.head()

,name,group,country,committee,email,phone,twitter,facebook
0,Adam GIEREK,Group of the Progressive Alliance of Socialist...,"[Poland, Unia, Pracy]","[ITRE, Committee on Industry, Research and Ene...",adam.gierek@europarl.europa.eu,+32(0)2 28 45781,https://twitter.com/adamgierek,https://www.facebook.com/AdamGierek
1,Ádám KÓSA,Group of the European People's Party (Christia...,"[Hungary, Fidesz-Magyar, Polgári, Szövetség-Ke...","[EMPL, Committee on Employment and Social Affa...",adam.kosa@europarl.europa.eu,+32(0)2 28 45208,http://twitter.com/adamkosamep,https://www.facebook.com/adamkosamep
2,Adam SZEJNFELD,Group of the European People's Party (Christia...,"[Poland, Platforma, Obywatelska]","[INTA, Committee on International Trade, DCAR,...",adam.szejnfeld@europarl.europa.eu,+32(0)2 28 45776,https://twitter.com/szejnfeld,https://www.facebook.com/PoselSzejnfeld/
3,Adina-Ioana VĂLEAN,Group of the European People's Party (Christia...,"[Romania, Partidul, Naţional, Liberal]","[ENVI, Committee on the Environment, Public He...",adinaioana.valean@europarl.europa.eu,+32(0)2 28 45861,https://twitter.com/AdinaValean,https://www.facebook.com/adina.valean
4,Agnes JONGERIUS,Group of the Progressive Alliance of Socialist...,"[Netherlands, Partij, van, de, Arbeid]","[EMPL, Committee on Employment and Social Affa...",agnes.jongerius@europarl.europa.eu,+32(0)2 28 45699,https://twitter.com/a_jongerius,https://www.facebook.com/PvdAEuropeesParlement


In [191]:
df.to_excel("EU_MEPs.xlsx")

In [195]:
751 - df.replace(-999, np.nan).count()

name           0
group          0
country        0
committee      2
email          0
phone          0
twitter      235
facebook     189
dtype: int64

In [3]:
url = "http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=3780&lang=en"

In [6]:
resp = requests.get(url)

In [7]:
soup = BeautifulSoup(resp.text, "lxml")

In [8]:
tab = soup.findAll("table", attrs={"cellpadding":"0", 
                                   "cellspacing":"0", "width":"100%"})[3].findAll("table")[0]

In [13]:
countries = [re.sub(" Permanent Representation of ", "",x.text) for x in tab.findAll("a")]
countries

['Belgium',
 'Bulgaria',
 'the Czech Republic',
 'Denmark',
 'Germany',
 'Estonia',
 'Ireland',
 'Greece',
 'Spain',
 'France',
 'Croatia',
 'Italy',
 'Cyprus',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Hungary',
 'Malta',
 'the Netherlands',
 'Austria',
 'Poland',
 'Portugal',
 'Romania',
 'Slovenia',
 'Slovakia',
 'Finland',
 'Sweden',
 'the United Kingdom']

In [14]:
links = ["http://europa.eu/whoiswho/public/"+x.attrs["href"] for x in tab.findAll("a")]
links

['http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=3781&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=209923&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=172741&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=3849&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=3901&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=172505&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=4251&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=644&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=4083&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=4180&lang=en',
 'http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hier

In [23]:
data = defaultdict(dict)

for country, link in zip(countries, links):
    country_resp = requests.get(link)
    country_soup = BeautifulSoup(country_resp.text, "lxml")
    
    country_tab = country_soup.findAll("table", attrs={"cellpadding":"0", 
                                "cellspacing":"0", "width":"100%"})[3].findAll("table")[0]
    
    representatives_names = [x.text for x in country_tab.findAll("a")]
    representatives_links = [x.attrs["href"] for x in country_tab.findAll("a")]
    
    data[country] = {"representatives": representatives_names,
                    "representative_links": representatives_links}

In [24]:
data

defaultdict(dict,
            {'Austria': {'representative_links': ['index.cfm?fuseaction=idea.hierarchy&nodeID=26969&personID=30233&lang=en',
               'index.cfm?fuseaction=idea.hierarchy&nodeID=26969&personID=297543&lang=en',
               'index.cfm?fuseaction=idea.hierarchy&nodeID=26969&personID=55864&lang=en',
               'index.cfm?fuseaction=idea.hierarchy&nodeID=26969&personID=115292&lang=en'],
              'representatives': ['MARSCHIK Nikolaus',
               'OBERREITER Thomas',
               'KMENTT Alexander',
               'PAIER Alexander']},
             'Belgium': {'representative_links': ['index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=77988&lang=en',
               'index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=113102&lang=en',
               'index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=149701&lang=en',
               'index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=154301&lang=en'],
              'representati

In [25]:
data["Belgium"]

{'representative_links': ['index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=77988&lang=en',
  'index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=113102&lang=en',
  'index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=149701&lang=en',
  'index.cfm?fuseaction=idea.hierarchy&nodeID=3781&personID=154301&lang=en'],
 'representatives': ['ROUX François',
  'HOOGMARTENS Jan',
  "CORNET D'ELZIUS François",
  'ROSSION Stéphanie']}

In [98]:
def get_info(url):
    
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "lxml")
    tab = soup.findAll("table", attrs={"cellpadding":"0", 
                                   "cellspacing":"0", "width":"100%"})[3].findAll("table")
    
    position = soup.findAll("td", {"itemprop":"jobTitle"})[0].text
    email = re.findall('href="mailto:(.*?)">', resp.text)[0]
    address = re.sub("[\s+|/]", " ",soup.findAll("span", {"itemprop":"address"})[0].text)
    telephone = soup.findAll("span", {"itemprop":"telephone"})[0].text

    return telephone, address, email, position

In [99]:
get_info("http://europa.eu/whoiswho/public/index.cfm?fuseaction=idea.hierarchy&nodeID=4180&personID=469427&lang=en")

('+32 22298211* ',
 ' Place de Louvain 14 Leuvenseplein 14  1000           Bruxelles Brussel  Belgique*   ',
 'courrier.bruxelles-dfra@diplomatie.gouv.fr',
 'Ambassador Extraordinary and PlenipotentiaryPermanent Representative (Coreper II)')

In [101]:
lst = []

for country in data:
       
    links = data[country]["representative_links"]
    names = data[country]["representatives"]
    
    for link, name in zip(links, names):
        
        point = {"Country":country, "Name":name, "Link":link}
        
        telephone, address, email, position = get_info("http://europa.eu/whoiswho/public/"+link)
        
        point["telephone"] = telephone
        point["address"] = address
        point["email"] = email
        point["position"] = position
        
        lst.append(point)

In [153]:
df = pd.DataFrame(lst)

In [154]:
df["address"] = df["address"].str.replace("  ", ", ").str.replace("(, )+", ", "
                ).str.replace("*","").str.replace("Bruxelles Brussel.*Belgique.*",
                "Bruxelles, Belgium").str.replace(",,", ",").str.strip()

In [155]:
df.to_excel("Permanent_representatives.xlsx")